In [1]:
import pandas as pd 

import requests
import json
from typing import Tuple

In [2]:
filename = "gift_list.csv"
df = pd.read_csv(filename)

## MY Link==PP, TW Link==TWpromotion, SIP==OLD, new discount < 10%

## preprocess

In [3]:
df["MY discount"] = df["NEW Discount"].apply(lambda x: 0) # float(x.strip("%")))
df["MY webprice"] = df["PP"]
df["MY equal"] = [False for i in range(len(df))]
df["MY error"] = [False for i in range(len(df))]

df["TW webprice"] = df["PP"]
df["TW equal"] = [False for i in range(len(df))]
df["TW error"] = [False for i in range(len(df))]

## helpers

In [4]:
def parse_id(url: str) -> Tuple[str, str]:
    shopid, itemid = url.split("-")[-1].split(".")[1:]
    return shopid, itemid

In [21]:
# for i in range(len(df)):
for i in df.index:
    if i % 100 == 0:
        print("iter {}...".format(i), end="\r")
    
    # MY
    url = df["MY Link"][i]
    shopid, itemid = parse_id(url)
    try:
        r = requests.get("https://shopee.com.my/api/v2/item/get?shopid={}&itemid={}".format(shopid, itemid))
        item = json.loads(r.content)["item"]
        models = item["models"]

        equal = False
        for m in models:
            p = m["price"] * 10e-6

            if abs(df["PP"][i] - p) < 10e-5:
                equal = True
                break
            else:
                # print(df["PP"][i], p)
                pass

        models = list(map(lambda m: "{}: {:.2f}".format(m["name"], m["price"]*10e-6), models))
        models = ", ".join(models)
        df.loc[i, "MY webprice"] = models

        df.loc[i, "MY equal"] = equal
        df.loc[i, "MY discount"] = item["show_discount"]
        
    except:
        df.loc[i, "MY error"] = True
        df.loc[i, "MY webprice"] = 0
        df.loc[i, "MY equal"] = False
        df.loc[i, "MY discount"] = 0
    
    # TW
    url = df["TW link"][i]
    shopid, itemid = parse_id(url)
    try:
        r = requests.get("https://shopee.tw/api/v2/item/get?shopid={}&itemid={}".format(shopid, itemid))
        models = json.loads(r.content)["item"]["models"]

        equal = False
        for m in models:
            p = int(m["price"] * 10e-6)

            if abs(df["TWpromotion price"][i] - p) < 10e-5:
                equal = True
                break
            else:
                # print(df["TWpromotion price"][i], p)
                pass

        models = list(map(lambda m: "{}: {:.2f}".format(m["name"], m["price"]*10e-6), models))
        models = ", ".join(models)
        df.loc[i, "TW webprice"] = models

        df.loc[i, "TW equal"] = equal
        
    except:
        df.loc[i, "TW error"] = True
        df.loc[i, "TW webprice"] = 0
        df.loc[i, "TW equal"] = False

/Users/tushicheng/py/lib/python3.7/site-packages/pandas/core/indexing.py:965: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self.obj[item] = s


## filter

In [8]:
full_filename = filename.strip(".csv") + "_full.csv"
df.to_csv(full_filename)

In [24]:
df_filter = df[
    (df["TW equal"] == False) |
    (df["MY equal"] == False) |
    (df["MY error"] == True) |
    (df["TW error"] == True) |
    ( (df["SIP"] == "OLD") & (df["MY discount"] <= 10) )
]

In [27]:
filtered_filename = filename.strip(".csv") + "_filtered.csv"

df_filter.to_csv(filtered_filename)

In [ ]:
MY discount 
MY webprice 
MY equal 
MY error 

TW webprice 
TW equal 
TW error